In [1]:
import pandas as pd
import json

In [3]:
df1 = pd.io.json.json_normalize(json.load(open("../data/madness2018.json")))
df2 = pd.io.json.json_normalize(json.load(open("../data/fsmadness2017.json")))
df3 = pd.io.json.json_normalize(json.load(open("../data/madness19-final.json")))

In [4]:
matches1 = pd.io.json.json_normalize(df1["tournament.matches"][0])
matches2 = pd.io.json.json_normalize(df2["tournament.matches"][0])
matches3 = pd.io.json.json_normalize(df3["tournament.matches"][0])
matches = pd.concat([matches1, matches2, matches3])

In [5]:
participants1 = pd.io.json.json_normalize(df1["tournament.participants"][0])
participants2 = pd.io.json.json_normalize(df2["tournament.participants"][0])
participants3 = pd.io.json.json_normalize(df3["tournament.participants"][0])
participants = pd.concat([participants1, participants2, participants3])

In [6]:
matches.to_csv("../results/all-matches-final.csv", index=False)

In [7]:
participants.to_csv("../results/all-participants-final.csv", index=False)

In [8]:
matches = matches[["match.player1_id", "match.player2_id", "match.scores_csv"]]

In [9]:
participants = participants[["participant.id", "participant.name", "participant.seed"]]

In [10]:
player1_joined = matches.merge(participants, left_on = "match.player1_id", right_on="participant.id")
player1_joined = player1_joined[["participant.name", "participant.seed", "match.scores_csv", "match.player2_id"]]
player1_joined = player1_joined.rename(columns = {"participant.name": "player_1_name",
                                 "participant.seed": "player_1_seed"})

In [11]:
all_matches = player1_joined.merge(participants, left_on = "match.player2_id", right_on="participant.id")
all_matches = all_matches[["player_1_name", "player_1_seed", "participant.name", "participant.seed", "match.scores_csv"]]
all_matches = all_matches.rename(columns = {"participant.name": "player_2_name",
                                 "participant.seed": "player_2_seed"})

In [12]:
all_matches = all_matches.dropna(subset=["player_1_name"])

In [13]:
all_matches["player_1_name"].append(all_matches["player_2_name"]).unique()

array(['Pulp Fiction', 'Out of Sight', 'Goodfellas', 'Toy Story',
       "Schindler's List", 'Fight Club', 'Chungking Express',
       'Barton Fink', 'Jackie Brown', 'The Silence of the Lambs', 'Heat',
       'Boogie Nights', 'Terminator 2: Judgment Day',
       'Dazed and Confused', 'Malcolm X', 'The Big Lebowski',
       'Groundhog Day', "Miller's Crossing", 'Magnolia', 'Unforgiven',
       'Being John Malkovich', 'Trainspotting', 'The Matrix',
       'Hoop Dreams', 'Fargo', 'L. A. Confidential', 'Before Sunrise',
       'Eyes Wide Shut', 'Rushmore', 'Reservoir Dogs',
       'The Shawshank Redemption', 'Se7en', 'The Godfather',
       'The Apartment', 'Taxi Driver', 'Citizen Kane',
       'The Godfather: Part II', 'Alien', 'Mulholland Dr.', 'Casablanca',
       'Blue Velvet', 'Chinatown', 'Jaws', 'Raiders of the Lost Ark',
       '2001: A Space Odyssey', 'Raging Bull', 'Apocalypse Now',
       'Do The Right Thing', 'Back to the Future', 'Star Wars',
       "One Flew Over the Cuckoo's

In [42]:
alt_row = all_matches.loc[all_matches["player_1_name"] == "Iron Man vs. Spider-Man 2"]
alt_row = alt_row.replace("Iron Man vs. Spider-Man 2", "Spider-Man 2")
all_matches = all_matches.replace("Iron Man vs. Spider-Man 2", "Iron Man")
all_matches = all_matches.append(alt_row)

In [14]:
all_matches = all_matches.replace("Assass. of Jesse James...", "The Assassination of Jesse James by the Coward Robert Ford")

In [53]:
def play_in_candidates(df, original, candidate_1, candidate_2):
    if original in df["player_1_name"].values:
        alt_row = df.loc[df["player_1_name"] == original]
    else:
        alt_row = df.loc[df["player_2_name"] == original]
    alt_row = alt_row.replace(original, candidate_1)
    df = df.replace(original, candidate_2)
    df = df.append(alt_row)
    return df

In [57]:
all_matches = play_in_candidates(all_matches, "Ocean's Eleven v Hot Fuzz", "Ocean's Eleven", "Hot Fuzz")
all_matches = play_in_candidates(all_matches, "Once v Dancer in the Dark", "Once", "Dancer in the Dark")
all_matches = play_in_candidates(all_matches, "Donnie Darko v Am. Psycho", "Donnie Darko", "American Psycho")
all_matches = play_in_candidates(all_matches, "Anchorman v 40-Year-Old Virgin", "Anchorman", "40-Year-Old Virgin")

In [15]:
all_matches = all_matches.replace('Young Frankenstein / Blazing Saddles', "Young Frankenstein")

In [16]:
all_matches

,player_1_name,player_1_seed,player_2_name,player_2_seed,match.scores_csv
0,Pulp Fiction,1,Wild at Heart,64,87-13
1,Pulp Fiction,1,Saving Private Ryan,33,82-18
2,Out of Sight,32,Saving Private Ryan,33,41-59
3,Pulp Fiction,1,Toy Story,16,66-34
4,Pulp Fiction,1,Fight Club,8,72-28
5,Pulp Fiction,1,The Silence of the Lambs,4,66-34
6,Pulp Fiction,1,Fargo,3,46-54
7,Goodfellas,2,Fargo,3,45-55
8,Toy Story,16,The Sixth Sense,49,75-25
9,Toy Story,16,Schindler's List,17,60-40


In [66]:
def append_other(df, candidate_1, candidate_2):
    if candidate_1 in df["player_1_name"].values:
        alt_row = df.loc[df["player_1_name"] == candidate_1]
    else:
        alt_row = df.loc[df["player_2_name"] == candidate_1]
    alt_row = alt_row.replace(candidate_1, candidate_2)
    print(alt_row)
    df = df.append(alt_row)
    return df

In [68]:
all_matches = append_other(all_matches, "Moon", "Primer")

              player_1_name  player_1_seed player_2_name  player_2_seed  \
127  No Country For Old Men              1        Primer             64   

    match.scores_csv  
127                   


In [70]:
all_matches = append_other(all_matches, "Atonement", "Away From Her")

           player_1_name  player_1_seed  player_2_name  player_2_seed  \
143  There Will Be Blood              2  Away From Her             63   

    match.scores_csv  
143                   


In [72]:
all_matches = append_other(all_matches, "Let The Right One In", "Away From Her")

       player_1_name  player_1_seed  player_2_name  player_2_seed  \
149  Children of Men             10  Away From Her             55   

    match.scores_csv  
149                   


In [74]:
all_matches = append_other(all_matches, "Grizzly Man", "The Fog of War")

       player_1_name  player_1_seed   player_2_name  player_2_seed  \
129  Pan's Labyrinth             16  The Fog of War             49   

    match.scores_csv  
129                   


In [76]:
all_matches = append_other(all_matches, "In Bruges", "Collateral")

    player_1_name  player_1_seed player_2_name  player_2_seed match.scores_csv
150   Ratatouille             23    Collateral             42                 


In [78]:
all_matches = append_other(all_matches, "Minority Report", "A.I. Artificial Intelligence")

                    player_1_name  player_1_seed player_2_name  player_2_seed  \
156  A.I. Artificial Intelligence             27        Oldboy             38   

    match.scores_csv  
156                   


In [80]:
all_matches = append_other(all_matches, "Kill Bill Vol. 1", "Kill Bill Vol. 2")

        player_1_name  player_1_seed       player_2_name  player_2_seed  \
130  Kill Bill Vol. 2             17  Dancer in the Dark             48   
130  Kill Bill Vol. 2             17                Once             48   

    match.scores_csv  
130                   
130                   


In [82]:
all_matches = append_other(all_matches, "Fellowship of the Ring", "The Return of the King")

              player_1_name  player_1_seed player_2_name  player_2_seed  \
157  The Return of the King             11   City of God             54   

    match.scores_csv  
157                   


In [17]:
all_matches.to_csv("../results/all-matches-with-seeds-final.csv", index=False)